In [5]:
import pandas as pd

from data import Dataset, get_transfer_dataset, get_random_datasets
from learner import PredictorLearner, TransferLearner
import analysis as a

# define training set sizes to be used in experiments
TRAINING_SET_SIZES = [0.2, 0.4, 0.6, 0.8]

# initialise results columns
rq1_results_fields = ['mse_accuracy_tgt_no_cv', 'mape_accuracy_tgt_no_cv', 'mse_accuracy_tgt_cv', 'mape_accuracy_tgt_cv',
                     'mse_accuracy_trans_no_cv', 'mape_accuracy_trans_no_cv', 'mse_accuracy_trans_cv', 'mape_accuracy_trans_cv']
rq1_results = pd.DataFrame(columns=rq1_results_fields) # done
rq2_results_fields = ['mape_accuracy_pred_20pct', 'mape_accuracy_pred_40pct', 'mape_accuracy_pred_60pct', 
                     'mape_accuracy_pred_80pct', 'mape_accuracy_trans_20pct', 'mape_accuracy_trans_40pct', 
                     'mape_accuracy_trans_60pct',  'mape_accuracy_trans_80pct', 'mse_accuracy_pred_20pct', 
                     'mse_accuracy_pred_40pct', 'mse_accuracy_pred_60pct', 'mse_accuracy_pred_80pct',  
                     'mse_accuracy_trans_20pct', 'mse_accuracy_trans_40pct', 'mse_accuracy_trans_60pct', 
                     'mse_accuracy_trans_80pct']
rq2_results = pd.DataFrame(columns=rq2_results_fields)
rq3_results_fields = ['training_time_pred_20pct_no_cv', 'training_time_pred_40pct_no_cv', 'training_time_pred_60pct_no_cv', 
                     'training_time_pred_80pct_no_cv', 'training_time_pred_20pct_cv', 'training_time_pred_40pct_cv', 
                     'training_time_pred_60pct_cv', 'training_time_pred_80pct_cv', 'training_time_trans_20pct_no_cv', 
                     'training_time_trans_40pct_no_cv', 'training_time_trans_60pct_no_cv', 'training_time_trans_80pct_no_cv', 
                     'training_time_trans_20pct_cv', 'training_time_trans_40pct_cv', 'training_time_trans_60pct_cv', 
                     'training_time_trans_80pct_cv']
rq3_results = pd.DataFrame(columns=rq3_results_fields)

# randomly select a source and target dataset for each subject system
nodejs_datasets, poppler_datasets, x264_datasets, xz_datasets = get_random_datasets()
src_nodejs, tgt_nodejs = nodejs_datasets
src_poppler, tgt_poppler = poppler_datasets
src_x264, tgt_x264 = x264_datasets
src_xz, tgt_xz = xz_datasets

src_x264.get_dataset()

,configurationID,cabac,ref,deblock,analyse,me,subme,mixed_ref,me_range,trellis,...,scenecut,rc_lookahead,mbtree,qpmax,aq-mode,size,frames,fps,kbs,time
0,1,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,3346923,209,932.51,1975.49,1.269
1,2,1,0,1,4,0,1,0,0,0,...,1,6,0,0,1,3737464,209,333.01,2206.00,2.839
2,11,0,0,1,0,0,2,0,0,0,...,0,6,0,0,0,2909433,209,592.72,1717.26,1.967
3,101,1,1,1,2,1,4,1,0,1,...,0,5,1,0,1,2265396,209,161.94,1337.13,6.149
4,102,1,1,1,2,1,4,1,0,1,...,0,0,1,0,0,2148516,209,200.57,1268.14,5.420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,96,1,1,1,2,1,6,0,0,0,...,1,1,1,0,1,2215668,209,162.90,1307.77,6.598
197,97,1,1,1,2,1,3,1,0,0,...,1,1,1,0,1,2146689,209,152.02,1267.06,5.433
198,98,1,1,0,2,1,3,0,1,0,...,1,2,1,0,1,2155032,209,210.55,1271.98,4.336
199,99,1,3,1,2,1,4,1,0,1,...,1,2,1,0,1,2301935,209,140.92,1358.69,8.815


In [ ]:
### RQ1

mape_accuracy_pred_no_cv = []
mape_accuracy_trans_no_cv = []
mape_accuracy_pred_cv = []
mape_accuracy_trans_cv = []
mse_accuracy_pred_no_cv = []
mse_accuracy_trans_no_cv = []
mse_accuracy_pred_cv = []
mse_accuracy_trans_cv = []

predictor = PredictorLearner()
transferrer = TransferLearner()

# get optimised predictor model using hyperparameter optimisation
X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
optimised_model = predictor.get_optimal_params(X_validate, y_validate)

# get accuracy of optimised model for predictor learner
predictor.fit(X_train, y_train, premade_model=optimised_model)
mape_accuracy_pred_cv.append(predictor.get_error(X_train, y_train, measure='mape'))
mse_accuracy_pred_cv.append(predictor.get_error(X_train, y_train, measure='mse'))

# get accuracy of non-optimised model for predictor learner
predictor.fit(X_train, y_train)
mape_accuracy_pred_no_cv.append(predictor.get_error(X_train, y_train, measure='mape'))
mse_accuracy_pred_no_cv.append(predictor.get_error(X_train, y_train, measure='mse'))


# get optimised transfer model using hyperparameter optimisation
X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, dataset_x264_target)
optimised_model = transferrer.get_optimal_params(X_validate, y_validate)

# get accuracy of optimised model for transfer learner
transferrer.fit(X_train, y_train, premade_model=optimised_model)
mape_accuracy_trans_cv.append(transferrer.get_error(X_train, y_train, measure='mape'))
mse_accuracy_trans_cv.append(predictor.get_error(X_train, y_train, measure='mse'))

# get accuracy of non-optimised model for transfer learner
transferrer.fit(X_train, y_train)
mape_accuracy_trans_no_cv.append(transferrer.get_error(X_train, y_train, measure='mape'))
mse_accuracy_trans_no_cv.append(predictor.get_error(X_train, y_train, measure='mse'))


rq1_results['mse_accuracy_tgt_no_cv'] = mse_accuracy_pred_no_cv
rq1_results['mape_accuracy_tgt_no_cv'] = mape_accuracy_pred_no_cv
rq1_results['mse_accuracy_tgt_cv'] = mse_accuracy_pred_cv
rq1_results['mape_accuracy_tgt_cv'] = mape_accuracy_pred_cv
rq1_results['mse_accuracy_trans_no_cv'] = mse_accuracy_trans_no_cv
rq1_results['mape_accuracy_trans_no_cv'] = mape_accuracy_trans_no_cv
rq1_results['mse_accuracy_trans_cv'] = mse_accuracy_trans_cv
rq1_results['mape_accuracy_trans_cv'] = mape_accuracy_trans_cv

In [ ]:
### RQ2
mape_accuracy_pred_20pct = []
mape_accuracy_pred_40pct = []
mape_accuracy_pred_60pct = []
mape_accuracy_pred_80pct = []
mape_accuracy_trans_20pct = []
mape_accuracy_trans_40pct = []
mape_accuracy_trans_60pct = []
mape_accuracy_trans_80pct = []
mse_accuracy_pred_20pct = []
mse_accuracy_pred_40pct = []
mse_accuracy_pred_60pct = []
mse_accuracy_pred_80pct = []
mse_accuracy_trans_20pct = []
mse_accuracy_trans_40pct = []
mse_accuracy_trans_60pct = []
mse_accuracy_trans_80pct = []

transferrer = TransferLearner()
predictor = PredictorLearner()

for train_size in TRAINING_SET_SIZES: 
    
    X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
    optimised_model = predictor.get_optimal_params(X_validate, y_validate)
    
    # get accuracy of predictor model for current training set size
    predictor.fit(X_train, y_train, premade_model=optimised_model)
    mape_accuracy_pred = predictor.get_error(X_train, y_train, measure='mape')
    mse_accuracy_pred = predictor.get_error(X_train, y_train, measure='mse')
    
    
    X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, 
                                                                    dataset_x264_target, 
                                                                    train_size=train_size)
    optimised_model = transferrer.get_optimal_params(X_validate, y_validate)
    
    # get accuracy of transfer model for each training set size
    transferrer.fit(X_train, y_train, premade_model=optimised_model)
    mape_accuracy_trans = transferrer.get_error(X_train, y_train, measure='mape')
    mse_accuracy_trans = transferrer.get_error(X_train, y_train, measure='mse')
    

    # record accuracy in appropriate results column
    if train_size == 0.2:
        mape_accuracy_pred_20pct.append(mape_accuracy_pred)
        mape_accuracy_trans_20pct.append(mape_accuracy_trans)
        mse_accuracy_pred_20pct.append(mse_accuracy_pred)
        mse_accuracy_trans_20pct.append(mse_accuracy_trans)
    elif train_size == 0.4:
        mape_accuracy_pred_40pct.append(mape_accuracy_pred)
        mape_accuracy_trans_40pct.append(mape_accuracy_trans)
        mse_accuracy_pred_40pct.append(mse_accuracy_pred)
        mse_accuracy_trans_40pct.append(mse_accuracy_trans)
    elif train_size == 0.6:
        mape_accuracy_pred_60pct.append(mape_accuracy_pred)
        mape_accuracy_trans_60pct.append(mape_accuracy_trans)
        mse_accuracy_pred_60pct.append(mse_accuracy_pred)
        mse_accuracy_trans_60pct.append(mse_accuracy_trans)
    elif train_size == 0.8:
        mape_accuracy_pred_80pct.append(mape_accuracy_pred)
        mape_accuracy_trans_80pct.append(mape_accuracy_trans)
        mse_accuracy_pred_80pct.append(mse_accuracy_pred)
        mse_accuracy_trans_80pct.append(mse_accuracy_trans)
        
        
rq2_results['mape_accuracy_pred_20pct'] = mape_accuracy_pred_20pct
rq2_results['mape_accuracy_pred_40pct'] = mape_accuracy_pred_40pct
rq2_results['mape_accuracy_pred_60pct'] = mape_accuracy_pred_60pct
rq2_results['mape_accuracy_pred_80pct'] = mape_accuracy_pred_80pct
rq2_results['mse_accuracy_pred_20pct'] = mse_accuracy_pred_20pct
rq2_results['mse_accuracy_pred_40pct'] = mse_accuracy_pred_40pct
rq2_results['mse_accuracy_pred_60pct'] = mse_accuracy_pred_60pct
rq2_results['mse_accuracy_pred_80pct'] = mse_accuracy_pred_80pct
rq2_results['mape_accuracy_trans_20pct'] = mape_accuracy_trans_20pct
rq2_results['mape_accuracy_trans_40pct'] = mape_accuracy_trans_40pct
rq2_results['mape_accuracy_trans_60pct'] = mape_accuracy_trans_60pct
rq2_results['mape_accuracy_trans_80pct'] = mape_accuracy_trans_80pct
rq2_results['mse_accuracy_trans_20pct'] = mse_accuracy_trans_20pct
rq2_results['mse_accuracy_trans_40pct'] = mse_accuracy_trans_40pct
rq2_results['mse_accuracy_trans_60pct'] = mse_accuracy_trans_40pct
rq2_results['mse_accuracy_trans_80pct'] = mse_accuracy_trans_80pct

In [ ]:
### RQ3

training_time_pred_20pct_no_cv = []
training_time_pred_40pct_no_cv = []
training_time_pred_60pct_no_cv = []
training_time_pred_80pct_no_cv = []
training_time_pred_20pct_cv = []
training_time_pred_40pct_cv = []
training_time_pred_60pct_cv = []
training_time_pred_80pct_cv = []
training_time_trans_20pct_no_cv = []
training_time_trans_40pct_no_cv = []
training_time_trans_60pct_no_cv = []
training_time_trans_80pct_no_cv = []
training_time_trans_20pct_cv = []
training_time_trans_40pct_cv = []
training_time_trans_60pct_cv = []
training_time_trans_80pct_cv = []

transferrer = TransferLearner()
predictor = PredictorLearner()

for train_size in TRAINING_SET_SIZES:
    
    X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
    # get optimised predictor model using hyperparameter optimisation
    optimised_model = predictor.get_optimal_params(X_validate, y_validate)
    predictor.fit(X_train, y_train, premade_model=optimised_model)
    
    # gather results
    training_time_pred_no_cv = predictor.get_training_time()
    training_time_pred_cv = predictor.get_training_time(include_optimisation_time=True)
    

    X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, dataset_x264_target)
    # get optimised transfer model using hyperparameter optimisation
    optimised_model = transferrer.get_optimal_params(X_validate, y_validate)
    transferrer.fit(X_train, y_train, premade_model=optimised_model)
    
    # gather results
    training_time_trans_no_cv = transferrer.get_training_time()
    training_time_trans_cv = transferrer.get_training_time(include_optimisation_time=True)


    # record accuracy in appropriate results column
    if train_size == 0.2:
        training_time_pred_20pct_no_cv.append(training_time_pred_no_cv)
        training_time_pred_20pct_cv.append(training_time_pred_cv)
        training_time_trans_20pct_no_cv.append(training_time_trans_no_cv)
        training_time_trans_20pct_cv.append(training_time_trans_cv)
    elif train_size == 0.4:
        training_time_pred_40pct_no_cv.append(training_time_pred_no_cv)
        training_time_pred_40pct_cv.append(training_time_pred_cv)
        training_time_trans_40pct_no_cv.append(training_time_trans_no_cv)
        training_time_trans_40pct_cv.append(training_time_trans_cv)
    elif train_size == 0.6:
        training_time_pred_60pct_no_cv.append(training_time_pred_no_cv)
        training_time_pred_60pct_cv.append(training_time_pred_cv)
        training_time_trans_60pct_no_cv.append(training_time_trans_no_cv)
        training_time_trans_60pct_cv.append(training_time_trans_cv)
    elif train_size == 0.8:
        training_time_pred_80pct_no_cv.append(training_time_pred_no_cv)
        training_time_pred_80pct_cv.append(training_time_pred_cv)
        training_time_trans_80pct_no_cv.append(training_time_trans_no_cv)
        training_time_trans_80pct_cv.append(training_time_trans_cv)
        
        
rq3_results['training_time_pred_20pct_no_cv'] = training_time_pred_20pct_no_cv
rq3_results['training_time_pred_40pct_no_cv'] = training_time_pred_40pct_no_cv
rq3_results['training_time_pred_60pct_no_cv'] = training_time_pred_60pct_no_cv
rq3_results['training_time_pred_80pct_no_cv'] = training_time_pred_80pct_no_cv
rq3_results['training_time_pred_20pct_cv'] = training_time_pred_20pct_cv
rq3_results['training_time_pred_40pct_cv'] = training_time_pred_40pct_cv
rq3_results['training_time_pred_60pct_cv'] = training_time_pred_60pct_cv
rq3_results['training_time_pred_80pct_cv'] = training_time_pred_80pct_cv
rq3_results['training_time_trans_20pct_no_cv'] = training_time_trans_20pct_no_cv
rq3_results['training_time_trans_40pct_no_cv'] = training_time_trans_40pct_no_cv
rq3_results['training_time_trans_60pct_no_cv'] = training_time_trans_60pct_no_cv
rq3_results['training_time_trans_80pct_no_cv'] = training_time_trans_80pct_no_cv
rq3_results['training_time_trans_20pct_cv'] = training_time_trans_20pct_cv
rq3_results['training_time_trans_40pct_cv'] = training_time_trans_40pct_cv
rq3_results['training_time_trans_60pct_cv'] = training_time_trans_60pct_cv
rq3_results['training_time_trans_80pct_cv'] = training_time_trans_80pct_cv

In [2]:
get_random_datasets()[0][1].get_dataset()

,configurationID,jitless,experimental-wasm-modules,experimental-vm-modules,preserve-symlinks-main,no-warnings,node-memory-debug,ops
0,0,0,0,0,0,0,0,13.541140
1,1,1,1,0,1,0,1,13.623182
2,10,1,0,1,1,1,0,13.655568
3,11,1,0,1,1,1,1,13.658441
4,12,0,0,1,1,1,1,13.408390
5,13,1,0,1,0,1,1,13.624011
6,14,0,1,1,1,0,0,13.582441
7,15,0,0,1,0,0,1,13.525642
8,16,1,1,1,1,0,1,13.666943
9,17,0,1,0,1,0,1,13.551160
